In [30]:
import os

from platform import python_version
import warnings
import time
import datetime as dt
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import multiprocessing as mp
import shutil

from imblearn.keras import BalancedBatchGenerator
from imblearn.under_sampling import RandomUnderSampler

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.densenet import DenseNet121, preprocess_input, decode_predictions
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.utils import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.initializers import *
from tensorflow.keras import regularizers

import pandas as pd
import numpy as np
import seaborn as sns
from plotly.subplots import make_subplots
import plotly.graph_objects as go

from PIL import Image
import xml.etree.ElementTree as ET
import psutil
import random

warnings.filterwarnings("ignore")
%matplotlib inline

In [31]:
img_size = 96
epochs = 75
batch_size = 32
testsplit = .25
targetx = 96
targety = 96
learning_rate = 0.0001
classes = 7
seed = 23
print(seed)


23


In [32]:
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define two different ImageDataGenerators with their respective directories
datagen1 = ImageDataGenerator(
        rescale=1./255,
        brightness_range=[0.9,1.1],
        horizontal_flip=True,
        fill_mode='nearest',
        validation_split=testsplit,
        preprocessing_function=preprocess_input
)

datagen2 = ImageDataGenerator(
        rescale=1./255,
        brightness_range=[0.9,1.1],
        horizontal_flip=True,
        fill_mode='nearest',
        validation_split=testsplit,
        preprocessing_function=preprocess_input
)

train_dir1 = 'KDEFMF/Male'
train_dir2 = 'KDEFMF/Female'

# Create two separate generators using different directories
train_generator1 = datagen1.flow_from_directory(
    train_dir1,
    target_size=(targetx,targety),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    shuffle=True,
    seed=seed
)
validation_generator1 = datagen1.flow_from_directory(
    'KDEFMF/Male',
    target_size=(targetx,targety),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
)
validation_generator2 = datagen2.flow_from_directory(
    'KDEFMF/Female',
    target_size=(targetx,targety),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
)

train_generator2 = datagen2.flow_from_directory(
    train_dir2,
    target_size=(targetx,targety),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    shuffle=True,
    seed=seed
)

# Combine labels from both generators
all_labels = list(train_generator1.classes) + list(train_generator2.classes)

# Get class indices (assuming they are the same for both generators)
class_indices = train_generator1.class_indices
index_to_class = {v: k for k, v in class_indices.items()}

# Count the number of occurrences of each class
class_counts = Counter(all_labels)

# Print the number of images per class
for class_index, count in class_counts.items():
    class_name = index_to_class[class_index]
    print(f"Class '{class_name}' has {count} images.")
    
def combined_generator(gen1, gen2, num_batches):
    while True:
        try:
            batch1_x, batch1_y = next(gen1)
            batch2_x, batch2_y = next(gen2)

            combined_x = np.concatenate((batch1_x, batch2_x), axis=0)
            combined_y = np.concatenate((batch1_y, batch2_y), axis=0)

            yield combined_x, combined_y
        except StopIteration as e:
            print("Generator exhausted or encountered an error:", e)
            break


Found 1124 images belonging to 7 classes.
Found 370 images belonging to 7 classes.
Found 358 images belonging to 7 classes.
Found 1086 images belonging to 7 classes.
Class 'angry' has 316 images.
Class 'disgust' has 316 images.
Class 'fear' has 316 images.
Class 'happy' has 316 images.
Class 'neutral' has 316 images.
Class 'sad' has 315 images.
Class 'surprise' has 315 images.


In [33]:
base_model = DenseNet121(include_top=False, weights='imagenet', input_shape=(targetx, targety, 3))

model = Sequential()

# Custom Layers neccesarry to acheive high acurracy 
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Flatten() (x)
x = BatchNormalization()(x)
x=Dropout(0.3)(x)
x = Dense(512, activation='relu',  kernel_initializer=glorot_uniform(seed),kernel_regularizer=regularizers.L2(0.001),activity_regularizer=regularizers.L2(1e-4), bias_initializer='zeros')(x)
x = BatchNormalization()(x)
x=Dropout(0.3)(x)
x = Dense(256, activation='relu',  kernel_initializer=glorot_uniform(seed),kernel_regularizer=regularizers.L2(0.001),activity_regularizer=regularizers.L2(1e-4), bias_initializer='zeros')(x)
x = BatchNormalization()(x)
x=Dropout(0.3)(x)
x = Dense(128, activation='relu',  kernel_initializer=glorot_uniform(seed),kernel_regularizer=regularizers.L2(0.001),activity_regularizer=regularizers.L2(1e-4), bias_initializer='zeros')(x)
x = BatchNormalization()(x)
x=Dropout(0.3)(x)
predictions = Dense(classes, activation='softmax', kernel_initializer='random_uniform', bias_initializer='zeros')(x)

model = tf.keras.Model(inputs=base_model.input, outputs=predictions)
optimizer = Adam(learning_rate=learning_rate)
loss = "categorical_crossentropy"

for layer in model.layers:
    layer.trainable = True

model.compile(optimizer=optimizer,
              loss=loss,
              metrics=["accuracy"])


In [34]:
# Define the number of batches you want from each generator
num_batches_per_gen = 100  # Adjust as needed

# Use the combined generator for training
combined_train_generator = combined_generator(train_generator1, train_generator2, num_batches_per_gen)

# Use the combined generator for validation
combined_validation_generator = combined_generator(validation_generator1, validation_generator2, num_batches_per_gen)


steps_per_epoch = total_train_samples // (2 * 32)
validation_steps = total_validation_samples // (2 * 32)

# Ensure correct optimizer and loss function
optimizer = Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Ensure ModelCheckpoint is correctly configured used for callback to save the best value based on val_accuracy
checkpoint = ModelCheckpoint('SeqBalancedmodel.keras', monitor='val_accuracy', save_best_only=True, verbose=1)

# Train the model with callbacks
history = model.fit(
    combined_train_generator,
    steps_per_epoch=steps_per_epoch,
    validation_data=combined_validation_generator,
    validation_steps=validation_steps,
    epochs=epochs,
    callbacks=[checkpoint]
)

Epoch 1/75



KeyboardInterrupt



In [ ]:
# Collect predictions and true labels
y_true = []
y_pred = []

for batch_x, batch_y in combined_validation_generator:
    # Make predictions
    predictions = model.predict(batch_x)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(batch_y, axis=1)
    
    # Append to lists
    y_true.extend(true_classes)
    y_pred.extend(predicted_classes)
    
    # Stop if you have processed the entire validation set
    if len(y_true) >= total_validation_samples:
        break

# Generate classification report
report = classification_report(y_true, y_pred, target_names=validation_generator1.class_indices.keys())
print(report)

In [ ]:
import tensorflow as tf
model =tf.keras.models.load_model('SeqBalancedmodel.keras')
predictions = model.predict(validation_generator1, steps=len(validation_generator1))
y = np.argmax(predictions, axis=1)

print('Classification Report')
cr = classification_report(y_true=validation_generator1.classes, y_pred=y, target_names=validation_generator1.class_indices)
print(cr)

In [ ]:
import tensorflow as tf
model =tf.keras.models.load_model('SeqBalancedmodel.keras')
predictions = model.predict(validation_generator2, steps=len(validation_generator2))
y = np.argmax(predictions, axis=1)

print('Classification Report')
cr = classification_report(y_true=validation_generator2.classes, y_pred=y, target_names=validation_generator2.class_indices)
print(cr)